In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import dmc
from process import *

In [4]:
evaluation_sets = ['rawMirrored', 'rawLinearSample']
data = processed_data()

In [5]:
train, test = split_data_by_id(data, 'rawMirrored')

In [6]:
len(train), len(test)

(947782, 236498)

In [7]:
ensemble = dmc.ensemble.Ensemble(train, test)

In [10]:
split = ensemble.splits['unknown_articleID-known_customerID-unknown_voucherID-known_productGroup']

In [14]:
for k in ensemble.splits:
    print(len(ensemble.splits[k][1]), k)

3239 unknown_articleID-known_customerID-unknown_voucherID-known_productGroup
6496 known_articleID-known_customerID-unknown_voucherID-known_productGroup
22160 unknown_articleID-unknown_customerID-known_voucherID-known_productGroup
14 unknown_articleID-unknown_customerID-known_voucherID-unknown_productGroup
2082 unknown_articleID-unknown_customerID-unknown_voucherID-known_productGroup
62265 unknown_articleID-known_customerID-known_voucherID-known_productGroup
12 unknown_articleID-known_customerID-known_voucherID-unknown_productGroup
42844 known_articleID-unknown_customerID-known_voucherID-known_productGroup
93197 known_articleID-known_customerID-known_voucherID-known_productGroup
4189 known_articleID-unknown_customerID-unknown_voucherID-known_productGroup


In [15]:
ensemble.transform??

In [ ]:
ensemble.classify??

In [19]:
ensemble.transform(binary_target=True, scalers=[dmc.transformation.normalize_raw_features]*10, 
                   ignore_features=[['orderID']] * 10)

KeyboardInterrupt: 